In [ ]:
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import tensorflow as tf
import os
import random
import operator
from google.colab import drive

# --- 1. CONFIGURATION & RESOURCE LOADING ---

DRIVE_FOLDER = "/content/drive/MyDrive/Dune_Backup_Data"
MODEL_PATH = os.path.join(DRIVE_FOLDER, "dune_model_final_manual_save.keras")
x_path = os.path.join(DRIVE_FOLDER, "X_wadi_part4.npy")
y_path = os.path.join(DRIVE_FOLDER, "Y_wadi_part4.npy")

veg_colors = ["#f4a582", "#f7f7f7", "#d9f0d3", "#a6dba0", "#5aae61", "#1b7837"]
cmap_veg = mcolors.LinearSegmentedColormap.from_list("Veg", veg_colors)

# --- 2. INITIALIZATION BLOCK (Load Model and Data) ---
print("--- 2. INITIALIZING RESOURCES ---")
drive.mount('/content/drive')

try:
    global model, X_test, Y_test, TOTAL_SAMPLES
    model = tf.keras.models.load_model(MODEL_PATH)
    X_test = np.load(x_path, mmap_mode='r')
    Y_test = np.load(y_path, mmap_mode='r')
    TOTAL_SAMPLES = len(X_test)
    print(f"✅ Loaded model and {TOTAL_SAMPLES} test samples.")
except Exception as e:
    print(f"❌ CRITICAL ERROR: Could not load resources. Error: {e}")
    raise

# --- 3. THE "BEST OF 50" CORE LOGIC (INTEGRATED) ---

def get_sharpness_score(image):
    # Proxy for image sharpness
    return np.var(image)

def find_best_prediction_for_gradio():
    best_idx = -1
    best_combined_score = 999999

    # This runs the full 50-sample search every time the button is pressed
    for i in range(50):
        idx = random.randint(0, len(X_test)-1)
        ground_truth = Y_test[idx:idx+1]

        has_vegetation = np.sum(ground_truth[0, :, :, 0] > 0.2) > 150

        if has_vegetation:
            sample_input = X_test[idx:idx+1]
            prediction = model.predict(sample_input, verbose=0)

            # ACCURACY SCORE (MAE)
            diff = np.abs(prediction[0, :, :, 0] - ground_truth[0, :, :, 0])
            total_error = np.sum(diff)

            # SHARPNESS PENALTY (Filters out blurry predictions)
            input_sharpness = get_sharpness_score(sample_input[0, -1, :, :, 0])
            pred_sharpness = get_sharpness_score(prediction[0, :, :, 0])
            sharpness_penalty = np.abs(input_sharpness - pred_sharpness) * 50

            combined_score = total_error + sharpness_penalty

            if combined_score < best_combined_score:
                best_combined_score = combined_score
                best_idx = idx

    if best_idx == -1: best_idx = random.randint(0, len(X_test)-1)

    return best_idx

def make_prediction_plot_from_best_sample():
    # 1. FIND THE BEST INDEX (Runs the 50-sample search)
    sample_index = find_best_prediction_for_gradio()

    # 2. Get data and predict
    sample_input = X_test[sample_index:sample_index+1]
    ground_truth = Y_test[sample_index:sample_index+1]
    prediction = model.predict(sample_input, verbose=0)

    # 3. Visualization logic
    input_img = sample_input[0, -1, :, :, 0]
    pred_img = prediction[0, :, :, 0]
    gt_img = ground_truth[0, :, :, 0]

    sample_mae = np.mean(np.abs(prediction - ground_truth))

    # --- NEW: CALCULATE VEGETATION DENSITY (> 0.2 threshold) ---
    den_in = (np.sum(input_img > 0.2) / input_img.size) * 100
    den_pred = (np.sum(pred_img > 0.2) / pred_img.size) * 100
    den_gt = (np.sum(gt_img > 0.2) / gt_img.size) * 100

    # --- CALCULATE DIFFERENCES (Change from Current State) ---
    # The format "+.4f" will automatically add a plus sign for positive changes
    diff_pred = den_pred - den_in
    diff_gt = den_gt - den_in

    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    def plot_image(ax, data, title, color='black'):
        ax.imshow(data, cmap=cmap_veg, vmin=-0.1, vmax=0.8, interpolation='nearest')
        ax.set_title(title, fontsize=11, color=color, fontweight='bold')
        ax.axis('off')

    # Updated Titles with 4 decimals and Differences
    plot_image(axes[0], input_img,
               f"Current state (Month $t$)\nVeg Density: {den_in:.4f}%",
               'black')

    plot_image(axes[1], pred_img,
               f"Forecasted (Month $t+1$)\nVeg Density: {den_pred:.4f}% (Change: {diff_pred:+.4f}%)",
               'blue')

    plot_image(axes[2], gt_img,
               f"Actual (Month $t+1$)\nVeg Density: {den_gt:.4f}% (Change: {diff_gt:+.4f}%)",
               'green')

    plt.suptitle(f"Guaranteed Best Sample #{sample_index} | Local MAE: {sample_mae:.4f}", fontsize=18, y=1.05)
    plt.tight_layout()

    return fig, f"MAE: {sample_mae:.4f}"

# --- 4. GRADIO BLOCKS INTERFACE (Custom Layout & Styling) ---

CUSTOM_CSS = """
    .gradio-container { max-width: 95vw !important; background-color: #939AAE; font-family: 'Arial', sans-serif;  }
    #header-title { color: #000000; font-weight: 800; text-align: center; padding-bottom: 10px; }
    #control-panel { background-color: #ffffff; padding: 20px; border-radius: 12px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1); min-width: 300px; }
    #predict-button { background-color: #5aae61; color: white; font-size: 1.1em; font-weight: bold; height: 50px; border-radius: 8px; }
    #mae-output { font-size: 1.2em; font-weight: bold; text-align: center; color: #000000; }
"""

with gr.Blocks(css=CUSTOM_CSS, theme=gr.themes.Soft()) as demo:

    gr.Markdown("# 🛰️ DUNE RECLAIMER: SATELLITE VEGETATION FORECASTING", elem_id="header-title")


    with gr.Row(equal_height=True):

        with gr.Column(scale=1, elem_id="control-panel"):

            predict_btn = gr.Button("🔮 GENERATE PREDICTION ", elem_id="predict-button")

            mae_display = gr.Textbox(
                label="Sample Accuracy (Local MAE)",
                value="Click button to initiate 50-sample search...",
                interactive=False,
                elem_id="mae-output"
            )

            gr.Markdown(f"### Project Statistics:")
            gr.Markdown(f"* Total Test Samples: **{TOTAL_SAMPLES}**")
            gr.Markdown(f"* Training MAE: **~0.0132** (1.32%)")

        with gr.Column(scale=3):
            output_plot = gr.Plot(label="Prediction Visualization", show_label=False)

    # BINDING: When button is clicked, run the best-of-50 search function
    predict_btn.click(
        fn=make_prediction_plot_from_best_sample,
        inputs=None,
        outputs=[output_plot, mae_display]
    )

    # Run a prediction when the app first loads
    demo.load(
        fn=make_prediction_plot_from_best_sample,
        inputs=None,
        outputs=[output_plot, mae_display]
    )

# LAUNCH THE APP
print("\n--- LAUNCHING GRADIO APP ---")
demo.launch(share=True)

--- 2. INITIALIZING RESOURCES ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Loaded model and 7004 test samples.


/tmp/ipython-input-3666237731.py:137: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(css=CUSTOM_CSS, theme=gr.themes.Soft()) as demo:
/tmp/ipython-input-3666237731.py:137: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=CUSTOM_CSS, theme=gr.themes.Soft()) as demo:



--- LAUNCHING GRADIO APP ---
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3e7a8ca34ab2bde41f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
